In [ ]:
# test pi_0
import os
import gym
import torch
import pprint
import argparse
import numpy as np
from torch.utils.tensorboard import SummaryWriter

from tianshou.env import DummyVectorEnv
from tianshou.utils.net.common import Net
from tianshou.trainer import offpolicy_trainer    ##
from tianshou.data import Collector, ReplayBuffer ##
from tianshou.utils.net.continuous import Actor, ActorProb, Critic


#-------------------------------------------------------------
# The belows are special for policy distral
from distral_offpolicy_trainer import Distral_offpolicy_trainer
####from distral_collector import Distral_Collector    # 惊了，这个也不需要
from distral_task_policy import TaskPolicy
from distral_distilled_policy import DistilledPolicy
#-------------------------------------------------------------



def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--task', type=str, default='Swimmer-v2')  # ‘Pendulum-v0’
    parser.add_argument('--seed', type=int, default=0)
    parser.add_argument('--buffer-size', type=int, default=20000)
    parser.add_argument('--actor-lr', type=float, default=3e-4)
    parser.add_argument('--critic-lr', type=float, default=1e-3)
    parser.add_argument('--il-lr', type=float, default=1e-3)
    parser.add_argument('--gamma', type=float, default=0.99)
    parser.add_argument('--tau', type=float, default=0.005)
    parser.add_argument('--alpha', type=float, default=0.2)
    parser.add_argument('--epoch', type=int, default=20)
    parser.add_argument('--step-per-epoch', type=int, default=2400)
    parser.add_argument('--collect-per-step', type=int, default=10)
    parser.add_argument('--batch-size', type=int, default=128)
    parser.add_argument('--layer-num', type=int, default=1)
    parser.add_argument('--training-num', type=int, default=8)
    parser.add_argument('--test-num', type=int, default=100)
    parser.add_argument('--logdir', type=str, default='log')
    parser.add_argument('--render', type=float, default=0.)
    parser.add_argument('--rew-norm', type=int, default=1)
    parser.add_argument('--ignore-done', type=int, default=1)
    parser.add_argument('--n-step', type=int, default=4)
    parser.add_argument(
        '--device', type=str,
        default='cuda' if torch.cuda.is_available() else 'cpu')
    args = parser.parse_known_args()[0]
    return args


args = get_args()
np.random.seed(args.seed)
torch.manual_seed(args.seed)
#args.step_per_epoch = 800
args.epoch = 3

torch.set_num_threads(1)  # we just need only one thread for NN
env = gym.make(args.task)
if args.task == 'Pendulum-v0':
    env.spec.reward_threshold = -250
args.state_shape = env.observation_space.shape or env.observation_space.n
args.action_shape = env.action_space.shape or env.action_space.n
args.max_action = env.action_space.high[0]
# you can also use tianshou.env.SubprocVectorEnv
# train_envs = gym.make(args.task)

# model 1

train_envs_1 = DummyVectorEnv(
    [lambda: gym.make(args.task) for _ in range(args.training_num)])
# test_envs = gym.make(args.task)
test_envs_1 = DummyVectorEnv(
    [lambda: gym.make(args.task) for _ in range(args.test_num)])

# seed
train_envs_1.seed(args.seed)
test_envs_1.seed(args.seed)

net_1 = Net(args.layer_num, args.state_shape, device=args.device)
actor_1 = ActorProb(
    net_1, args.action_shape, args.max_action, args.device, unbounded=True
).to(args.device)
actor_optim_1 = torch.optim.Adam(actor_1.parameters(), lr=args.actor_lr)
net_c1_1 = Net(args.layer_num, args.state_shape,
               args.action_shape, concat=True, device=args.device)
critic1_1 = Critic(net_c1_1, args.device).to(args.device)
critic1_optim_1 = torch.optim.Adam(critic1_1.parameters(), lr=args.critic_lr)
net_c2_1 = Net(args.layer_num, args.state_shape,
               args.action_shape, concat=True, device=args.device)
critic2_1 = Critic(net_c2_1, args.device).to(args.device)
critic2_optim_1 = torch.optim.Adam(critic2_1.parameters(), lr=args.critic_lr)
policy_1 = TaskPolicy(
    actor_1, actor_optim_1, critic1_1, critic1_optim_1, critic2_1, critic2_optim_1,
    action_range=[env.action_space.low[0], env.action_space.high[0]],
    tau=args.tau, gamma=args.gamma, alpha=args.alpha,
    reward_normalization=args.rew_norm,
    ignore_done=args.ignore_done,
    estimation_step=args.n_step)

# model 2

train_envs_2 = DummyVectorEnv(
    [lambda: gym.make(args.task) for _ in range(args.training_num)])
# test_envs = gym.make(args.task)
test_envs_2 = DummyVectorEnv(
    [lambda: gym.make(args.task) for _ in range(args.test_num)])

# seed
train_envs_2.seed(args.seed)
test_envs_2.seed(args.seed)

net_2 = Net(args.layer_num, args.state_shape, device=args.device)
actor_2 = ActorProb(
    net_2, args.action_shape, args.max_action, args.device, unbounded=True
).to(args.device)
actor_optim_2 = torch.optim.Adam(actor_2.parameters(), lr=args.actor_lr)
net_c1_2 = Net(args.layer_num, args.state_shape,
               args.action_shape, concat=True, device=args.device)
critic1_2 = Critic(net_c1_2, args.device).to(args.device)
critic1_optim_2 = torch.optim.Adam(critic1_2.parameters(), lr=args.critic_lr)
net_c2_2 = Net(args.layer_num, args.state_shape,
               args.action_shape, concat=True, device=args.device)
critic2_2 = Critic(net_c2_2, args.device).to(args.device)
critic2_optim_2 = torch.optim.Adam(critic2_2.parameters(), lr=args.critic_lr)
policy_2 = TaskPolicy(
    actor_2, actor_optim_2, critic1_2, critic1_optim_2, critic2_2, critic2_optim_2,
    action_range=[env.action_space.low[0], env.action_space.high[0]],
    tau=args.tau, gamma=args.gamma, alpha=args.alpha,
    reward_normalization=args.rew_norm,
    ignore_done=args.ignore_done,
    estimation_step=args.n_step)


# collector
train_collector_1 = Collector(
    policy_1, train_envs_1, ReplayBuffer(args.buffer_size))
test_collector_1 = Collector(policy_1, test_envs_1)
train_collector_2 = Collector(
    policy_2, train_envs_2, ReplayBuffer(args.buffer_size))
test_collector_2 = Collector(policy_2, test_envs_2)
# train_collector.collect(n_step=args.buffer_size)
# log
log_path = os.path.join(args.logdir, args.task, 'sac_distral')
writer = SummaryWriter(log_path)


def save_fn(policy):
    torch.save(policy.state_dict(), os.path.join(log_path, 'policy.pth'))


def stop_fn(mean_rewards):
    return mean_rewards >= env.spec.reward_threshold


for itr in range(10):
    if itr == 0:
        # train policy 1
        result = offpolicy_trainer(
                policy_1, train_collector_1, test_collector_1, args.epoch,
                args.step_per_epoch, args.collect_per_step, args.test_num,
                args.batch_size, stop_fn=stop_fn, save_fn=save_fn, writer=writer)
        if __name__ == '__main__':
            pprint.pprint(result)
            # Let's watch its performance!
            policy_1.eval()
            collector = Collector(policy_1, env)
            result = collector.collect(n_episode=1, render=args.render)
            print(f'Final reward: {result["rew"]}, length: {result["len"]}')
        
        # train policy 2
        result = offpolicy_trainer(
                policy_2, train_collector_2, test_collector_2, args.epoch,
                args.step_per_epoch, args.collect_per_step, args.test_num,
                args.batch_size, stop_fn=stop_fn, save_fn=save_fn, writer=writer)
        if __name__ == '__main__':
            pprint.pprint(result)
            # Let's watch its performance!
            policy_2.eval()
            collector = Collector(policy_2, env)
            result = collector.collect(n_episode=1, render=args.render)
            print(f'Final reward: {result["rew"]}, length: {result["len"]}')
        
        # train distilled policy
        if args.task == 'Pendulum-v0':
            env.spec.reward_threshold = -300  # lower the goal
        net = ActorProb(
                Net(1, args.state_shape, device=args.device),
                args.action_shape, args.max_action, args.device
                ).to(args.device)
        
        optim = torch.optim.Adam(net.parameters(), lr=args.il_lr)

        distilled_policy = DistilledPolicy(net, optim, 
                    action_range=[env.action_space.low[0], env.action_space.high[0]],
                            mode='continuous')

        distilled_policy_test_collector = Collector(
                    distilled_policy,
                    DummyVectorEnv(
                        [lambda: gym.make(args.task) for _ in range(args.test_num)])
                )

        train_collector_1.reset()
        train_collector_2.reset()

        result = Distral_offpolicy_trainer(
                    distilled_policy, train_collector_1, train_collector_2, 
                    distilled_policy_test_collector, 20,
                    args.step_per_epoch // 5, args.collect_per_step, args.test_num,
                    args.batch_size, stop_fn=stop_fn, save_fn=save_fn, writer=writer)
        if __name__ == '__main__':
            pprint.pprint(result)
            # Let's watch its performance!
            distilled_policy.eval()
            collector = Collector(distilled_policy, env)
            result = collector.collect(n_episode=1, render=args.render)
            print(f'Final reward: {result["rew"]}, length: {result["len"]}')
    else:
        # train policy 1
        result = offpolicy_trainer(
                policy_1, train_collector_1, test_collector_1, args.epoch,
                args.step_per_epoch, args.collect_per_step, args.test_num,
                args.batch_size, stop_fn=stop_fn, save_fn=save_fn, writer=writer)
        if __name__ == '__main__':
            pprint.pprint(result)
            # Let's watch its performance!
            policy_1.eval()
            collector = Collector(policy_1, env)
            result = collector.collect(n_episode=1, render=args.render)
            print(f'Final reward: {result["rew"]}, length: {result["len"]}')
        # train policy 2
        result = offpolicy_trainer(
                policy_2, train_collector_2, test_collector_2, args.epoch,
                args.step_per_epoch, args.collect_per_step, args.test_num,
                args.batch_size, stop_fn=stop_fn, save_fn=save_fn, writer=writer)
        if __name__ == '__main__':
            pprint.pprint(result)
            # Let's watch its performance!
            policy_2.eval()
            collector = Collector(policy_2, env)
            result = collector.collect(n_episode=1, render=args.render)
            print(f'Final reward: {result["rew"]}, length: {result["len"]}')
        # train distilled policy
        train_collector_1.reset()
        train_collector_2.reset()
        
        result = Distral_offpolicy_trainer(
                    distilled_policy, train_collector_1, train_collector_2, 
                    distilled_policy_test_collector, 20,
                    args.step_per_epoch // 5, args.collect_per_step, args.test_num,
                    args.batch_size, stop_fn=stop_fn, save_fn=save_fn, writer=writer)
        if __name__ == '__main__':
            pprint.pprint(result)
            # Let's watch its performance!
            distilled_policy.eval()
            collector = Collector(distilled_policy, env)
            result = collector.collect(n_episode=1, render=args.render)
            print(f'Final reward: {result["rew"]}, length: {result["len"]}')

Epoch #1: 2401it [00:27, 88.67it/s, env_step=24000, len=1000, loss/actor=-19.829323, loss/critic1=57.594335, loss/critic2=57.912906, n/ep=8, n/st=8000, rew=28.19, v/ep=4.26, v/st=4261.11]                           
Epoch #2:   0%|          | 0/2400 [00:00<?, ?it/s]

Epoch #1: test_reward: 30.411748 ± 3.299431, best_reward: 30.411748 ± 3.299431 in #1


Epoch #2: 2401it [00:27, 87.19it/s, env_step=48000, len=1000, loss/actor=-24.099110, loss/critic1=4.999235, loss/critic2=5.078643, n/ep=8, n/st=8000, rew=39.19, v/ep=4.66, v/st=4659.87]                             
Epoch #3:   0%|          | 0/2400 [00:00<?, ?it/s]

Epoch #2: test_reward: 45.800676 ± 2.478960, best_reward: 45.800676 ± 2.478960 in #2


Epoch #3: 2401it [00:26, 89.05it/s, env_step=72000, len=1000, loss/actor=5.716048, loss/critic1=6.864888, loss/critic2=6.587452, n/ep=8, n/st=8000, rew=25.27, v/ep=4.32, v/st=4322.36]                             


...

Epoch #7: 481it [00:05, 84.50it/s, env_step=104000, len=1000, n/ep=8, n/st=8000, rew=37.39, v/ep=4.27, v/st=4270.46]                         
Epoch #8:   0%|          | 0/480 [00:00<?, ?it/s]

Epoch #7: test_reward: 41.747759 ± 1.995299, best_reward: 41.747759 ± 1.995299 in #7


Epoch #8: 481it [00:05, 86.02it/s, env_step=120000, len=1000, n/ep=8, n/st=8000, rew=35.91, v/ep=4.23, v/st=4230.25]                          
Epoch #9:   0%|          | 0/480 [00:00<?, ?it/s]

Epoch #8: test_reward: 36.438003 ± 1.204271, best_reward: 41.747759 ± 1.995299 in #7


Epoch #9: 481it [00:05, 85.25it/s, env_step=136000, len=1000, n/ep=8, n/st=8000, rew=37.42, v/ep=4.35, v/st=4347.42]                          
